In [ ]:
# Setup Tensorflow

import tensorflow as tf
gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
# Setup Local Libs

import sys
sys.path.append("../src/")

from utils import *
from training import *
from data_generators import *
from model_builders import *

In [ ]:
# Imports

from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import load_model, Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

In [ ]:
# Set model filepaths

bbox_model_filepath = "../experiments/results/grid_search/imgsize-224_convblocks-4_basefilters-16_densesize-1024/best_model.h5"
bbox_separable_model_filepath = "../experiments/results/separable_grid_search/imgsize-112_convblocks-3_basefilters-16_densesize-1024/best_model.h5"
triplet_extractor_model_filepath = "../experiments/results/triplet_base/dist-eucl/best_model.h5"
triplet_extractor_distances_filepath = "../experiments/results/triplet_base/dist-eucl/distances.csv"

In [ ]:
# Auxiliary function for camera demonstartion

def crop_face(frame, model, img_size=(224, 224), ret_color=(255, 0, 0), upper_text="Camera - Press Q to quit"):
    if (np.prod(frame.shape) != 0):
        aux_frame = cv2.cvtColor(cv2.resize(frame, img_size, interpolation=cv2.INTER_LINEAR), cv2.COLOR_BGR2RGB)
        aux_frame = (aux_frame/255.0)[np.newaxis, :, :, :]
        bbox = model.predict(aux_frame)[0]
        bbox[[0, 2]] *= float(frame.shape[1])
        bbox[[1, 3]] *= float(frame.shape[0])
        bbox = np.around(bbox).astype(np.int32)

        face = np.copy(frame[bbox[1]:bbox[3], bbox[0]:bbox[2]])
        cv2.rectangle(frame, tuple(bbox[:2]), tuple(bbox[2:]), ret_color, 2)

        cv2.imshow(upper_text, frame)
        cv2.imshow('Face', face)
    
    return face

def get_comparison_imgs(model, img_size=(224, 224)):
    comp_imgs = []
    cam = cv2.VideoCapture(0)
    print("Take 5 photos for comparison. Press F to take the photo")
    while(len(comp_imgs) < 5):
        _, frame = cam.read()
        face = crop_face(frame, model, img_size, upper_text="Press F to take the photo")
        if (cv2.waitKey(1) & 0xFF == ord('f')) and (face is not None):
            comp_imgs += [face]
            print("Picture Taken")
    cam.release()
    cv2.destroyAllWindows()

    print("Images For Comparison")
    fig, axs = plt.subplots(5, 1, figsize=(40, 20))
    for img, ax in zip(comp_imgs, axs):
        ax.axis('off')
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    return comp_imgs

def display_camera_bbox(model, img_size=(224, 224)):
    cam = cv2.VideoCapture(0)
    while(True):
        _, frame = cam.read()
        _ = crop_face(frame, model, img_size)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cam.release()
    cv2.destroyAllWindows()

def get_only_extractor(verf_model, extract_layer_name="vgg16_face_extractor/"):
    verf_model = verf_model.get_layer(name=extract_layer_name)
    verf_model = Model(verf_model.input, verf_model.output)
    return verf_model

def display_camera_verification(bbox_model, verf_model, comp_imgs,
                                bbox_img_size=(224, 224), verf_img_size=(224, 224),
                                dist_type='eucl', threshold=1.0,
                                preprocess_func=vgg16.preprocess_input,
                                verf_freq=25,
                                extract_layer_name="vgg16_face_extractor/"):
    
    verf_model = get_only_extractor(verf_model, extract_layer_name=extract_layer_name)
    comp_imgs = [cv2.resize(img, verf_img_size, interpolation=cv2.INTER_LINEAR) for img in comp_imgs]
    comp_imgs = np.stack([cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in comp_imgs]).astype(np.float32)
    comp_imgs = verf_model.predict(preprocess_func(comp_imgs))
    
    count, same = 0, False
    cam = cv2.VideoCapture(0)
    while(True):
        _, frame = cam.read()
        face = crop_face(frame, bbox_model, bbox_img_size, ret_color=((0, 255, 0) if same else (0, 0, 255)),
                        upper_text="Green=Same Person; Red=Different Person - Press Q to quit")
        if face is not None:
            if count < verf_freq:
                count += 1
            else:
                vect = cv2.cvtColor(cv2.resize(face, verf_img_size, interpolation=cv2.INTER_LINEAR), cv2.COLOR_BGR2RGB)
                vect = preprocess_func(vect.astype(np.float32))[np.newaxis, :, :, :]
                vect = verf_model.predict(vect)[0]
                
                same = np.sum(np.square(comp_imgs-vect), axis=1)
                same = np.count_nonzero(same < threshold) >= 3
                count = 0
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cam.release()
    cv2.destroyAllWindows()

In [ ]:
# Samples of Face Segmentation Using the Camera as Input

# Regular model
model = load_model(bbox_model_filepath, compile=False)
display_camera_bbox(model, img_size=(224, 224))
tf.keras.backend.clear_session()

# Model using Depth-Wise Separable Convolution
model = load_model(bbox_separable_model_filepath, compile=False)
display_camera_bbox(model, img_size=(112, 112))
tf.keras.backend.clear_session()

In [ ]:
# Samples of Face Verfication Using the Camera as Input

bbox_model = load_model(bbox_separable_model_filepath, compile=False)
comp_imgs = get_comparison_imgs(bbox_model, img_size=(112, 112))

In [ ]:
verf_thresh = pd.read_csv(triplet_extractor_distances_filepath)
verf_thresh = verf_thresh.loc[verf_thresh["Set"] == "Train"]
verf_thresh = float((verf_thresh["Pos Dist Mean"] + verf_thresh["Neg Dist Mean"]) / 2.0)
verf_model = load_model(triplet_extractor_model_filepath, compile=False, custom_objects={'L2Normalization':L2Normalization})
verf_model = build_triplet_classifier_model(verf_model, dist_type='eucl', threshold=verf_thresh)
display_camera_verification(bbox_model, verf_model, comp_imgs,
                            bbox_img_size=(112, 112), verf_img_size=(224, 224),
                            dist_type='eucl', threshold=verf_thresh)